# Assignment 3

## Test 3 methods for allocation

### Import necessary packages

In [1]:
import pandas as pd
import os
import ast
import numpy as np

In [2]:
def tier(row_num,name):
    each_dict_str=channel_spend.iloc[row_num,1]
    each_dict_dict=ast.literal_eval(each_dict_str)
    result=pd.Series(each_dict_dict,name=name)
    return result

# find the last interaction
def get_last_ele(x):  
    if x.last_valid_index() is np.nan:  # 一行中全为NaN
        return np.nan
    else:
        return x[x.last_valid_index()]

# Use recursion to find last non-direct clisk
def last_NDC(x,col_num):
    if x[col_num] is np.nan or x[col_num] == 'direct':  # 一行中全为NaN
        return last_NDC(x,col_num-1)
    else:
        return x[col_num]

### load and clean the data

In [3]:
# load data
# channel_spend
#os.chdir(r'D:\桌面的文件夹\读书\NYU\Marketing Analytics\Homework\HW3')
channel_spend = pd.read_csv('channel_spend_student_data.csv')
tier1=tier(0,'tier1')
tier2=tier(1,'tier2')
tier3=tier(2,'tier3')
channel_spend=pd.DataFrame([tier1,tier2,tier3])

attribution_allocation=pd.read_csv('attribution_allocation_student_data.csv')
attribution_allocation=attribution_allocation[attribution_allocation['convert_TF']==True]

### methods for allocation
• Last interaction

• Last non-direct interaction

• First interaction

• Linear

• Position Based

• Time Decay

• Data-driven (Google Analytics)


In this assignment, We would use the following 3 methods:

1. Last interaction

2. First interaction

3. Last Non-direct Click

### Method 1: Last interaction
this model uses the last touch as the channel, so we would like to first define a function that extract the the last non-NaN touch column, and then apply it to the whole dataset.

### Method 2: First interaction
because in the previous step, we have examined that there is no missing values in the column "touch1", so we could conveniently use the first column as the first interaction channel.

### Method 3: Last Non-direct Model
We choose this model because we think the last non-direct model is just as different as the last interaction model, since many customers would not directly pay when they see the ads or when referred by others; instead, these pieces of ads or information would remain in their memories so that whenever they need the products, those ads or information would jump out from their minds immediately. Therefore, we should focus on this model and try to strengthen the effectiveness of the last non-direct interaction. 

### create the columns to attribute

In [4]:
# last interaction
attribution_allocation['Last_Interaction']= attribution_allocation.iloc[:,1:6].apply(get_last_ele, axis=1)
# first interaction
attribution_allocation['First_Interaction']= attribution_allocation.touch1
# last non-direct click
attribution_allocation['Last_Non-Direct_Click']= attribution_allocation.apply(last_NDC, axis=1,args=(5,))

### make pivot_table

In [5]:
# last interaction
last_interaction = pd.pivot_table(attribution_allocation,values='convert_TF',index=['tier'],columns=['Last_Interaction'],aggfunc=np.sum)
last_interaction.columns.name = None
# first interaction
first_interaction = pd.pivot_table(attribution_allocation,values='convert_TF',index=['tier'],columns=['First_Interaction'],aggfunc=np.sum)
first_interaction.columns.name = None
# last non-direct click
last_non_direct_click = pd.pivot_table(attribution_allocation,values='convert_TF',index=['tier'],columns=['Last_Non-Direct_Click'],aggfunc=np.sum).iloc[:,1:]
last_non_direct_click.columns.name = None

### outcomes

In [6]:
last_interaction

,direct,display,email,organic_search,paid_search,referral,social
tier,,,,,,,
1,3.0,531.0,299.0,2.0,180.0,1187.0,622.0
2,NaN,824.0,532.0,3.0,340.0,2124.0,1069.0
3,3.0,959.0,686.0,5.0,484.0,2670.0,1256.0


In [7]:
first_interaction

,direct,display,email,organic_search,paid_search,referral,social
tier,,,,,,,
1,2.0,484.0,246.0,NaN,128.0,1493.0,471.0
2,3.0,735.0,430.0,5.0,247.0,2674.0,798.0
3,6.0,840.0,589.0,5.0,322.0,3313.0,988.0


In [8]:
last_non_direct_click

,display,email,organic_search,paid_search,referral,social
tier,,,,,,
1,531.0,299.0,2.0,180.0,1189.0,622.0
2,824.0,532.0,3.0,340.0,2124.0,1069.0
3,959.0,686.0,5.0,484.0,2671.0,1257.0


## Calculate CAC for Each Channel

In [9]:
# Adjust the order of the columns and change the indexes
channel_spend = channel_spend[['direct', 'display', 'email', 'organic_search', 'paid_search','referral', 'social']]
channel_spend.index=[1,2,3]
channel_spend

,direct,display,email,organic_search,paid_search,referral,social
1,0.0,1000.0,1000.0,0.0,1000.0,1000.0,1000.0
2,0.0,2000.0,2000.0,0.0,2000.0,2000.0,2000.0
3,0.0,3000.0,3000.0,0.0,3000.0,3000.0,3000.0


In [10]:
# last interaction
CAC_last_interaction = channel_spend.div(last_interaction,axis=1)
# first interaction
CAC_first_interaction = channel_spend.div(first_interaction,axis=1)
# last non-direct click
CAC_last_non_direct_click = channel_spend.div(last_non_direct_click,axis=1).iloc[:,1:]

### outcomes

In [11]:
CAC_last_interaction

,direct,display,email,organic_search,paid_search,referral,social
1,0.0,1.883239,3.344482,0.0,5.555556,0.842460,1.607717
2,NaN,2.427184,3.759398,0.0,5.882353,0.941620,1.870907
3,0.0,3.128259,4.373178,0.0,6.198347,1.123596,2.388535


In [12]:
CAC_first_interaction

,direct,display,email,organic_search,paid_search,referral,social
1,0.0,2.066116,4.065041,NaN,7.812500,0.669792,2.123142
2,0.0,2.721088,4.651163,0.0,8.097166,0.747943,2.506266
3,0.0,3.571429,5.093379,0.0,9.316770,0.905524,3.036437


In [13]:
CAC_last_non_direct_click

,display,email,organic_search,paid_search,referral,social
1,1.883239,3.344482,0.0,5.555556,0.841043,1.607717
2,2.427184,3.759398,0.0,5.882353,0.941620,1.870907
3,3.128259,4.373178,0.0,6.198347,1.123175,2.386635


## Insights

### Observations

1.Last interaction  

In this method, there are two totally free channels: direct and organic_search in which there is no CAC exists. Referral channel has the lowest CAC for all three tiers, following with social, display, email and paid_search. 

2.First interaction

In this method, there are two totally free channels: direct and organic_search in which there is no CAC exists. Referral channel has the lowest CAC for all three tiers; CAC for display channel is the second lowest for tier 1 customers, however, CAC of social channel becomes lower for tier 2 and 3 comparing with display channel. Moreover, email channel has the second highest CAC, just lower than paid_search, which has the highest CAC.


3.Last Non-direct Click

In this method, there are one free channel: organic_search in which there is no CAC exists since there is no direct channel using this method. Referral channel has the lowest CAC for all three tiers, following with social, email and paid_search. 

### Potential Conclusions

Among all these methods, the referral method is always the most important because not only it is the channel that occupies most customers and but also it is the cheapist among all paid channels, regardless of the tiers.This indicates that we should focus more on this channel to generate higher revenues. At the same time, we find that among all three methods, the number of customers through organic search and direct visit is quite low,so it suggests that we cannot rely on these free channels; instead, we should pay more attention to those paid channels.
In addition,(marginal CAC / increase in number of customers) from tier 1 to tier 2 is greatly lower than that from tier 2 to tier 3, indicating that we should invest and focus more on tier 2 customers who could maximize our effectiveness of marketing.



## We choose Last Interaction method

Considering the scenario, in most cases for a clothing business, the main intention of launching campaigns is to drive consumers to buy. Therefore, we choose the last interaction method.

| Display: Cumulative spend | Cumulative number of conversions | Average CAC | Marginal spend | Marginal Coversion | Marginal CAC
 :-: | :-: | :-: | :-: | :-: | :-:
| 1000 | 531 | 1.88 | 1000 | 531 | 1.88 |
| 2000 | 824 | 2.42 | 1000 | 293 | 3.41 |
| 3000 | 959 | 3.13 | 1000 | 135 | 7.41 |

| Email: Cumulative spend | Cumulative number of conversions | Average CAC | Marginal spend | Marginal Coversion | Marginal CAC |
 :-: | :-: | :-: | :-: | :-: | :-:
| 1000 | 299 | 3.34 | 1000 | 299 | 3.34 |
| 2000 | 532 | 3.76 | 1000 | 233 | 4.29 |
| 3000 | 686 | 4.37 | 1000 | 154 | 6.49 |

| Paid Search: Cumulative spend | Cumulative number of conversions | Average CAC | Marginal spend | Marginal Coversion | Marginal CAC |
 :-: | :-: | :-: | :-: | :-: | :-:
| 1000 | 180 | 5.56 |1000|180|5.56|
| 2000 | 340 | 5.88 | 1000 | 160 | 6.25 |
| 3000 | 484 | 6.20 | 1000 | 144 | 6.94 |

| Referral: Cumulative spend | Cumulative number of conversions | Average CAC | Marginal spend | Marginal Coversion | Marginal CAC |
 :-: | :-: | :-: | :-: | :-: | :-:
| 1000 | 1187 | 0.84 | 1000 | 1187 | 0.84 |
| 2000 | 2124 | 0.94 | 1000 | 937 | 1.06 |
| 3000 | 2670 | 1.12 | 1000 | 546 | 1.83 |

| Social: Cumulative spend | Cumulative number of conversions | Average CAC | Marginal spend | Marginal Coversion | Marginal CAC |
 :-: | :-: | :-: | :-: | :-: | :-:
|1000	|622	|1.61	|1000	|622	|1.61|
|2000	|1069	|1.87	|1000	|447	|2.24|
|3000	|1256	|2.39	|1000|	187	|5.35|

To allocate $11,000 we would like to find out the most valuable combination of without 11 lowest marginal CAC.

 || Display| Email | Paid Search | Referral| Social | 
:-: | :-: | :-: | :-: | :-: | :-:
|Marginal CAC_1	|1.88	|3.34	|5.56	|0.84	|1.61|
|Marginal CAC_2 |3.41	|4.29	|6.25	|1.07	|2.24|
|Marginal CAC_3	|7.41	|6.49	|6.94   |1.83	|5.35|

We decide to invest 3000 in referral, because Referral has the relative low CAC for all magrinal spend. We decide to invest 2000 dollars in display, 2000 dollars in Email,1000 dollars in Paid Search, 3000 dollars in Social.

 || Display| Email | Paid Search | Referral| Social | 
:-: | :-: | :-: | :-: | :-: | :-:
|First  1000|531	|299	|180	|1187	|622|
|Second 1000|293	|233	|       |937	|447|
|Third  1000|    	|    	|       |546	|187|

